### ____init____ialisieren der benötigten Klassen
Im nächsten Schritt werden die Klassen definiert, deren Objekte in der CSV beschrieben werden.

In [123]:
class Task:
    def __init__(self, ID, name, activities):
        self.ID = ID
        self.name = name
        self.activities = activities

class Activity:
    def __init__(self, ID, name, duration, parentTask, semaphoresIN, semaphoresOUT, mutexes):
        self.ID = ID
        self.name = name
        self.duration = duration
        self.parentTask = parentTask
        self.semaphoresIN = semaphoresIN
        self.semaphoresOUT = semaphoresOUT
        self.mutexes = mutexes
        self.currentValue = 0
        self.currentDuration = duration

class Mutex:
    def __init__(self, ID, name, activities):
        self.ID = ID
        self.name = name
        self.activities = activities

class Semaphore:
    def __init__(self, ID, name, groupWith, initialValue, activityIN, activityOUT):
        self.ID = ID
        self.name = name
        self.groupWith = groupWith
        self.initialValue = initialValue
        self.activityIN = activityIN
        self.activityOUT = activityOUT
        self.currentValue = 0

### Test der Klassen
Um die Klassen zu testen wird das folgende Diagramm aufgebaut: 

![Fehler beim Abrufen der Datei: ./img/test_diagram.png](./img/test_diagram.png)

In [ ]:
task1 = Task(1, "Task1", [1])
task2 = Task(2, "Task2", [2])
task3 = Task(3, "Task3", [3])
task4 = Task(4, "Task4", [4])
task5 = Task(5, "Task5", [5, 7])
task6 = Task(6, "Task6", [6])

activity1 = Activity(1, "Activity1", 1, [12, 13], [])
activity2 = Activity(2, "Activity2", 2, [24], [234])
activity3 = Activity(3, "Activity3", 3, [36], [234])
activity4 = Activity(4, "Activity4", 4, [46], [234])
activity5a = Activity(5, "Activity5a", 5, [55], [])
activity5b = Activity(7, "Activity5b", 5, [56], [])
activity6 = Activity(6, "Activity6", 6, [65], [])

mutex234 = Mutex(234, "Mutex234", [2, 3, 4])

semaphore12 = Semaphore(12, "Semaphore12", [2], None)
semaphore13 = Semaphore(13, "Semaphore13", [3], None)
semaphore24 = Semaphore(24, "Semaphore24", [4], None)
semaphore36 = Semaphore(36, "Semaphore36", [6], None)
semaphore46 = Semaphore(46, "Semaphore46", [6], None)
semaphore55 = Semaphore(55, "Semaphore55", [7], 1)
semaphore56 = Semaphore(56, "Semaphore56", [5], None)
semaphore65 = Semaphore(65, "Semaphore65", [5], 1)

### Einlesen der CSV Datei
Als nächstes sollen die Objekte aus einer CSV Datei eingelesen werden.

In [124]:
dataFilePath = "./dataTest2.csv"

data = []
linecells = []

import pandas as pd
df = pd.read_csv(dataFilePath, sep=',')

In [125]:
df

,Task_ID,Task_Name,Activity_ID,Activity_Name,Activity_Duration,Semaphore_ID,Semaphore_Name,Semaphore_Initial_Value,Predecessor_Semaphore_ID,Mutex_ID,Mutex_Name
0,1,Task1,1,Activity1,1,12;13;14,Semaphore12;Semaphore13;Semaphore14,0;1;1,21,None,None
1,2,Task2,2,Activity2,1,21,Semaphore21,0,[12;13];14,None,None


In [126]:
# split seperated values into list elements
df["Semaphore_ID"] = df["Semaphore_ID"].astype(str)
df["Semaphore_ID"] = df["Semaphore_ID"].str.split(";")
df["Semaphore_Name"] = df["Semaphore_Name"].astype(str)
df["Semaphore_Name"] = df["Semaphore_Name"].str.split(";")
df["Semaphore_Initial_Value"] = df["Semaphore_Initial_Value"].astype(str)
df["Semaphore_Initial_Value"] = df["Semaphore_Initial_Value"].str.split(";")
df["Predecessor_Semaphore_ID"] = df["Predecessor_Semaphore_ID"].astype(str)
df["Predecessor_Semaphore_ID"] = df["Predecessor_Semaphore_ID"].str.split(";")
df["Mutex_ID"] = df["Mutex_ID"].astype(str)
df["Mutex_ID"] = df["Mutex_ID"].str.split(";")
df["Mutex_Name"] = df["Mutex_Name"].astype(str)
df["Mutex_Name"] = df["Mutex_Name"].str.split(";")

In [127]:
df

,Task_ID,Task_Name,Activity_ID,Activity_Name,Activity_Duration,Semaphore_ID,Semaphore_Name,Semaphore_Initial_Value,Predecessor_Semaphore_ID,Mutex_ID,Mutex_Name
0,1,Task1,1,Activity1,1,"[12, 13, 14]","[Semaphore12, Semaphore13, Semaphore14]","[0, 1, 1]",[21],[None],[None]
1,2,Task2,2,Activity2,1,[21],[Semaphore21],[0],"[[12, 13], 14]",[None],[None]


In [128]:
# only during the creation process of the tree
tasks_IDs = []
tasks = []

activities_IDs = []
activities = []

semaphore_IDs = []
semaphores = []

mutex_IDs = []
mutexs = []

for index, row in df.iterrows():
    # first: instantiation of objects
    if row["Task_ID"] not in tasks_IDs:
        tasks_IDs.append(row["Task_ID"])
        tasks.append(Task(row["Task_ID"], row["Task_Name"], []))
    
    if row["Activity_ID"] not in activities_IDs:
        activities_IDs.append(row["Activity_ID"])
        activities.append(Activity(row["Activity_ID"], row["Activity_Name"], row["Activity_Duration"], [], row["Predecessor_Semaphore_ID"], [], []))
        print(row["Predecessor_Semaphore_ID"])
    
    # semaphoren anhand der VorgängerID erstellen, nicht anhand der herausgehenden ID
    for index, semaphore_ID in enumerate(row["Semaphore_ID"]):
        if semaphore_ID != "None" and semaphore_ID not in semaphore_IDs:
            semaphore_IDs.append(semaphore_ID)
            semaphores.append(Semaphore(semaphore_ID, row["Semaphore_Name"][index], [], int(row["Semaphore_Initial_Value"][index]), [], []))
        
    for index, mutex_ID in enumerate(row["Mutex_ID"]):
        if mutex_ID != "None" and mutex_ID not in mutex_IDs:
            mutex_IDs.append(mutex_ID)
            mutexs.append(Mutex(mutex_ID, row["Mutex_Name"][index], []))
            
    # add semaphoreIN to activities and semaphores
    
    # secondly: linking of objects
    tasks[tasks_IDs.index(row["Task_ID"])].activities.append(activities[activities_IDs.index(row["Activity_ID"])])
    
    activities[activities_IDs.index(row["Activity_ID"])].parentTask = tasks[tasks_IDs.index(row["Task_ID"])]
    for semaphore_ID in row["Semaphore_ID"]:
        if semaphore_ID != "None":
            activities[activities_IDs.index(row["Activity_ID"])].semaphoresOUT.append(semaphores[semaphore_IDs.index(semaphore_ID)])
            semaphores[semaphore_IDs.index(semaphore_ID)].activityOUT = activities[activities_IDs.index(row["Activity_ID"])]
    for mutex_ID in row["Mutex_ID"]:
        if mutex_ID != "None":
            activities[activities_IDs.index(row["Activity_ID"])].mutexes.append(mutexs[mutex_IDs.index(mutex_ID)])
            mutexs[mutex_IDs.index(mutex_ID)].activities.append(activities[activities_IDs.index(row["Activity_ID"])])


['21']
['[12', '13]', '14']


In [129]:
# add semaphoreIN to activities and semaphores
for activity in activities:
    buf = activity.semaphoresIN
    activity.semaphoresIN = []
    while len(buf) > 0:
        semaphore_id = buf.pop(0)
        # if start equals '[' -> start of or)
        if semaphore_id[0] == '[':
            semaphore_id = semaphore_id[1:]
            
            if semaphore_id[-1] == ']':
                semaphore_id = semaphore_id[:-1]
                activity.semaphoresIN.append(semaphores[semaphore_IDs.index(semaphore_id)])
                semaphores[semaphore_IDs.index(semaphore_id)].activityIN = activity
                print(semaphore_id)
            else:
                semaphore_ids = []
                while semaphore_id[-1] != ']':
                    semaphore_ids.append(semaphore_id)
                    semaphore_id = buf.pop(0)
                semaphore_ids.append(semaphore_id[:-1])
                for semaphore_id in semaphore_ids:
                    activity.semaphoresIN.append(semaphores[semaphore_IDs.index(semaphore_id)])
                    semaphores[semaphore_IDs.index(semaphore_id)].activityIN = activity
                    # set the groupWith value of the semaphores and add every semaphore to the group except for the current one
                    for semaphore in semaphore_ids:
                        if not semaphore == semaphore_id:
                            semaphores[semaphore_IDs.index(semaphore_id)].groupWith.append(semaphores[semaphore_IDs.index(semaphore)])
                print(semaphore_ids)
        
        # if end equals ']' -> end of or)
        
        # else -> and
        else:
            activity.semaphoresIN.append(semaphores[semaphore_IDs.index(semaphore_id)])
            semaphores[semaphore_IDs.index(semaphore_id)].activityIN = activity
            print(semaphore_id)

21
['12', '13']
14


In [40]:
activities
print("hallo")

hallo


In [8]:
root = tasks[0]
print("hallo")

hallo


In [68]:
print("Task 1 ist mit folgendem Task verbunden: Task " + str(tasks[0].activities[0].semaphoresOUT[0].activityIN.parentTask.ID))

Task 1 ist mit folgendem Task verbunden: Task 2


In [2]:
import graphviz as gv

dot = gv.Digraph(comment='Rectangle with Arrow')

dot.node('A', shape='rectangle', label='Rectangle A')
dot.node('B', shape='rectangle', label='Rectangle B')

dot.edge('A', 'B', arrowhead='vee')

dot.render('rectangle_arrow', view=True, format='jpg')

'rectangle_arrow.jpg.jpg'

In [3]:
dot = gv.Digraph(comment='Dashed Arrow')

dot.node('A', shape='rectangle', label='Rectangle A', style='filled', fillcolor='lightblue')
dot.node('B', shape='rectangle', label='Rectangle B', style='filled', fillcolor='lightgreen')

dot.edge('A', 'B', arrowhead='vee', style='dashed')

dot.render('rectangle_arrow', view=True, format='jpg')

'rectangle_arrow.jpg.jpg'

In [19]:
from graphviz import Digraph

dot = Digraph(comment='Rectangle with Dividing Line')

dot.node('A', shape='record', label='{Above Text|Below Text}')

dot.render('rectangle_arrow', view=True, format='jpg')

'rectangle_arrow.jpg'

In [130]:
import graphviz as gv
dot = gv.Digraph(comment='Flowchart')
dummyCounter = 0

In [131]:
# mmaybe pass ID of activity, semaphore that you want to color for the animation

ACTIVITY_RUNNING = 'green'
ACTIVITY_IDLE = 'white'

SEMAPHORE_ACTIVE = 'red'
SEMAPHORE_BLOCKING = 'black'

def createRects():
    for activity in activities:
        color = ACTIVITY_IDLE
        if activity.currentValue > 0:
            color = ACTIVITY_RUNNING
        dot.node(str(activity.ID), shape='record', style='rounded,filled', label='{'+activity.parentTask.name+'|'+activity.name+'}', fillcolor=color)
        
def createMutexs():
    for mutex in mutexs:
        dot.node(str(mutex.ID), shape='polygon', sides='5', label=mutex.name)
        for activity in mutex.activities:
            dot.edge(str(mutex.ID), str(activity.ID), arrowhead='none', style='dashed', splines='polyline')
        
def createSemaphores():
    global dummyCounter
    
    # add color just like in createRects
    
    semaphores_buf = semaphores.copy()
    while len(semaphores_buf) != 0:
        
        semaphore = semaphores_buf.pop(0)
        
        color = SEMAPHORE_BLOCKING
        if semaphore.currentValue > 0:
            color = SEMAPHORE_ACTIVE
            
        # if semaphore has groupWith, connect those semaphores together. Those will then be erased from the list
        if len(semaphore.groupWith) != 0:
            
            # TODO: add initial Value to the semaphore
            
            # draw semaphores
            middleDummyName = "Dummy" + str(dummyCounter)
            dot.node(middleDummyName, shape='point', width="0.01", height="0.01", color=color)
            
            # first the semaphore itself
            dot.edge(str(semaphore.activityOUT.ID), middleDummyName, label=semaphore.name, arrowhead='none', splines='polyline', color=color)
            dummyCounter += 1
            
            # after the groupSemaphores
            colorOfLastSemaphore = color
            for groupSemaphore in semaphore.groupWith:
                if groupSemaphore.currentValue > 0:
                    color = SEMAPHORE_ACTIVE
                    colorOfLastSemaphore = SEMAPHORE_ACTIVE
                else:
                    color = SEMAPHORE_BLOCKING
                
                dot.edge(str(groupSemaphore.activityOUT.ID), middleDummyName, label=groupSemaphore.name, arrowhead='none', splines='polyline', color=color)
                semaphores_buf.remove(groupSemaphore)
                
            dot.node(middleDummyName, shape='point', width="0.01", height="0.01", color=color)
            dot.edge(middleDummyName, str(semaphore.activityIN.ID), arrowhead='normal', splines='polyline', color=colorOfLastSemaphore)
        else:
            # if semaphore is leading to the same task where it came from, use another arrowhead
            if semaphore.activityIN.parentTask.ID == semaphore.activityOUT.parentTask.ID:
                # if semaphore has an initial value, connect an edge to the semaphore
                if semaphore.initialValue > 0:
                    dot.node("Dummy" + str(dummyCounter), shape='point', width="0.01", height="0.01")
                    dot.edge(str(semaphore.activityOUT.ID), "Dummy" + str(dummyCounter), arrowhead='none', splines='polyline', color=color)
                    dot.edge("Dummy" + str(dummyCounter), str(semaphore.activityIN.ID), label=semaphore.name, arrowhead='onormal', splines='polyline', color=color)
                    dummyCounter += 1
                    dot.node("Dummy" + str(dummyCounter), shape='point', xlabel=(lambda valorem: '' if valorem == 1 else str(valorem))(semaphore.initialValue))
                    dot.edge("Dummy" + str(dummyCounter-1), "Dummy" + str(dummyCounter), arrowhead='none', splines='polyline')
                    dummyCounter += 1
                else:
                    dot.edge(str(semaphore.activityOUT.ID), str(semaphore.activityIN.ID), label=semaphore.name, arrowhead='onormal', splines='polyline', color=color)
            else:
                if semaphore.initialValue > 0:
                    dot.node("Dummy" + str(dummyCounter), shape='point', width="0.01", height="0.01")
                    dot.edge(str(semaphore.activityOUT.ID), "Dummy" + str(dummyCounter), arrowhead='none', splines='polyline', color=color)
                    dot.edge("Dummy" + str(dummyCounter), str(semaphore.activityIN.ID), label=semaphore.name, splines='polyline', color=color)
                    dummyCounter += 1
                    dot.node("Dummy" + str(dummyCounter), shape='point', xlabel=(lambda valorem: '' if valorem == 1 else str(valorem))(semaphore.initialValue))
                    dot.edge("Dummy" + str(dummyCounter-1), "Dummy" + str(dummyCounter), arrowhead='none', splines='polyline')
                    dummyCounter += 1
                else:
                    dot.edge(str(semaphore.activityOUT.ID), str(semaphore.activityIN.ID), label=semaphore.name, splines='polyline', color=color)
            

In [10]:
createRects()
createMutexs()
createSemaphores()
dot.render('rectangle_arrow', view=True, format='jpg')

'rectangle_arrow.jpg'

In [132]:
historySemaphores = []

def getAllStartPoints():
    # get all activities that have no predecessor
    startPoints = []
    for semaphore in semaphores:
        if semaphore.initialValue > 0:
            startPoints.append(semaphore)
            
    return startPoints

def getActiveSemaphores():
    activeSemaphores = []
    for semaphore in semaphores:
        if semaphore.currentValue > 0:
            activeSemaphores.append(semaphore)
            
    return activeSemaphores

def getActiveActivities():
    activeActivities = []
    for activity in activities:
        if activity.currentValue > 0:
            activeActivities.append(activity)
            
    return activeActivities

def getNextFrame():
    # create a new image with highlighted nodes and edges (Tasks/Activities, Semaphores)
    startPoints = getAllStartPoints()
    
    # check if you can activate a new semaphore
    for startPoint in startPoints:
        if not startPoint.currentValue > 0:
            startPoint.currentValue = 1;
            startPoint.initialValue -= 1;
        # else: do nothing - wait for the semaphore to become free
        
    activeSemaphores = getActiveSemaphores()
        
    # check for the activities behind the active semaphores
    for activeSemaphore in activeSemaphores:
        # look, if the following activity is ready to run (all incoming semaphores == 1) or already running
        if not activeSemaphore.activityIN.currentValue > 0:
            # check if all incoming semaphores are active
            
            flag = True
            semaphoresIN = activeSemaphore.activityIN.semaphoresIN.copy()
            
            while flag and len(semaphoresIN) != 0:
                semaphoreIN = semaphoresIN.pop(0)
                # if there is a group, check if one of the semaphores is active
                if len(semaphoreIN.groupWith) != 0:
                    if not semaphoreIN.currentValue > 0:
                        for groupSemaphore in semaphoreIN.groupWith:
                            if groupSemaphore.currentValue > 0:
                                break
                            
                            if groupSemaphore == semaphoreIN.groupWith[-1]:
                                flag = False
                                break
                    if flag:
                        for groupSemaphore in semaphoreIN.groupWith:
                            semaphoresIN.remove(groupSemaphore)
                elif semaphoreIN.currentValue == 0:
                    flag = False
                    break
            
            if flag:
                activeSemaphore.activityIN.currentValue = 1
                
                
    activeActivities = getActiveActivities()
    
    for activeActivity in activeActivities:
        if activeActivity.currentDuration > 0: 
            activeActivity.currentDuration -= 1
        else:
            # next steps: activate all semaphores going out of the activity
            semaphoresIN = activeActivity.semaphoresIN
            for semaphoreIN in semaphoresIN:
                semaphoreIN.currentValue = 0
            activeActivity.currentValue = 0
            semaphoresOUT = activeActivity.semaphoresOUT
            for semaphoreOUT in semaphoresOUT:
                semaphoreOUT.currentValue = 1
                
            activeActivity.currentDuration = activeActivity.duration
            
    activeSemaphores = getActiveSemaphores()
        
    # check for the activities behind the active semaphores
    for activeSemaphore in activeSemaphores:
        # look, if the following activity is ready to run (all incoming semaphores == 1) or already running
        if not activeSemaphore.activityIN.currentValue > 0:
            # check if all incoming semaphores are active
            
            flag = True
            semaphoresIN = activeSemaphore.activityIN.semaphoresIN.copy()
            
            while flag and len(semaphoresIN) != 0:
                semaphoreIN = semaphoresIN.pop(0)
                # if there is a group, check if one of the semaphores is active
                if len(semaphoreIN.groupWith) != 0:
                    if not semaphoreIN.currentValue > 0:
                        for groupSemaphore in semaphoreIN.groupWith:
                            if groupSemaphore.currentValue > 0:
                                break
                            
                            if groupSemaphore == semaphoreIN.groupWith[-1]:
                                flag = False
                                break
                    if flag:
                        for groupSemaphore in semaphoreIN.groupWith:
                            semaphoresIN.remove(groupSemaphore)
                elif semaphoreIN.currentValue == 0:
                    flag = False
                    break
            
            if flag:
                activeSemaphore.activityIN.currentValue = 1
                

def saveInitialValuesSemaphores():
    # save the initial values of all semaphores
    for semaphore in semaphores:
        if semaphore.initialValue > 0:
            historySemaphores.append([semaphore.ID, semaphore.initialValue])

def reinitializeInitialValuesSemaphores():
    # set all semaphores back to their initial values
    for valorem in historySemaphores:
        semaphores[semaphore_IDs.index(valorem[0])].initialValue = valorem[1]

In [351]:
getAllActives()

([<__main__.Semaphore at 0x22ce6b53940>,
 [])

In [104]:
getNextFrame()

In [133]:
import graphviz as gv

# just a helper method
def createNextFrame():
    getNextFrame()
    getCurrentImage()
    
def getCurrentImage():
    global dot
    dot = gv.Digraph(comment='Flowchart')
    global dummyCounter
    dummyCounter = 0
    
    createRects()
    createMutexs()
    createSemaphores()
    dot.render('rectangle_arrow', view=True, format='svg')

In [506]:
class Task:
    def __init__(self, ID, name, activities):
        self.ID = ID
        self.name = name
        self.activities = activities

class Activity:
    def __init__(self, ID, name, duration, parentTask, semaphoresIN, semaphoresOUT, mutexes):
        self.ID = ID
        self.name = name
        self.duration = duration
        self.parentTask = parentTask
        self.semaphoresIN = semaphoresIN
        self.semaphoresOUT = semaphoresOUT
        self.mutexes = mutexes
        self.currentValue = 0
        self.currentDuration = duration

class Mutex:
    def __init__(self, ID, name, activities):
        self.ID = ID
        self.name = name
        self.activities = activities

class Semaphore:
    def __init__(self, ID, name, groupWith, initialValue, activityIN, activityOUT):
        self.ID = ID
        self.name = name
        self.groupWith = groupWith
        self.initialValue = initialValue
        self.activityIN = activityIN
        self.activityOUT = activityOUT
        self.currentValue = 0

dataFilePath = "./dataTest2.csv"

data = []
linecells = []

import pandas as pd
df = pd.read_csv(dataFilePath, sep=',')

# split seperated values into list elements
df["Semaphore_ID"] = df["Semaphore_ID"].astype(str)
df["Semaphore_ID"] = df["Semaphore_ID"].str.split(";")
df["Semaphore_Name"] = df["Semaphore_Name"].astype(str)
df["Semaphore_Name"] = df["Semaphore_Name"].str.split(";")
df["Semaphore_Initial_Value"] = df["Semaphore_Initial_Value"].astype(str)
df["Semaphore_Initial_Value"] = df["Semaphore_Initial_Value"].str.split(";")
df["Predecessor_Semaphore_ID"] = df["Predecessor_Semaphore_ID"].astype(str)
df["Predecessor_Semaphore_ID"] = df["Predecessor_Semaphore_ID"].str.split(";")
df["Mutex_ID"] = df["Mutex_ID"].astype(str)
df["Mutex_ID"] = df["Mutex_ID"].str.split(";")
df["Mutex_Name"] = df["Mutex_Name"].astype(str)
df["Mutex_Name"] = df["Mutex_Name"].str.split(";")

# only during the creation process of the tree
tasks_IDs = []
tasks = []

activities_IDs = []
activities = []

semaphore_IDs = []
semaphores = []

mutex_IDs = []
mutexs = []

for index, row in df.iterrows():
    # first: instantiation of objects
    if row["Task_ID"] not in tasks_IDs:
        tasks_IDs.append(row["Task_ID"])
        tasks.append(Task(row["Task_ID"], row["Task_Name"], []))
    
    if row["Activity_ID"] not in activities_IDs:
        activities_IDs.append(row["Activity_ID"])
        activities.append(Activity(row["Activity_ID"], row["Activity_Name"], row["Activity_Duration"], [], row["Predecessor_Semaphore_ID"], [], []))
        print(row["Predecessor_Semaphore_ID"])
    
    # semaphoren anhand der VorgängerID erstellen, nicht anhand der herausgehenden ID
    for index, semaphore_ID in enumerate(row["Semaphore_ID"]):
        if semaphore_ID != "None" and semaphore_ID not in semaphore_IDs:
            semaphore_IDs.append(semaphore_ID)
            semaphores.append(Semaphore(semaphore_ID, row["Semaphore_Name"][index], [], int(row["Semaphore_Initial_Value"][index]), [], []))
        
    for index, mutex_ID in enumerate(row["Mutex_ID"]):
        if mutex_ID != "None" and mutex_ID not in mutex_IDs:
            mutex_IDs.append(mutex_ID)
            mutexs.append(Mutex(mutex_ID, row["Mutex_Name"][index], []))
            
    # add semaphoreIN to activities and semaphores
    
    # secondly: linking of objects
    tasks[tasks_IDs.index(row["Task_ID"])].activities.append(activities[activities_IDs.index(row["Activity_ID"])])
    
    activities[activities_IDs.index(row["Activity_ID"])].parentTask = tasks[tasks_IDs.index(row["Task_ID"])]
    for semaphore_ID in row["Semaphore_ID"]:
        if semaphore_ID != "None":
            activities[activities_IDs.index(row["Activity_ID"])].semaphoresOUT.append(semaphores[semaphore_IDs.index(semaphore_ID)])
            semaphores[semaphore_IDs.index(semaphore_ID)].activityOUT = activities[activities_IDs.index(row["Activity_ID"])]
    for mutex_ID in row["Mutex_ID"]:
        if mutex_ID != "None":
            activities[activities_IDs.index(row["Activity_ID"])].mutexes.append(mutexs[mutex_IDs.index(mutex_ID)])
            mutexs[mutex_IDs.index(mutex_ID)].activities.append(activities[activities_IDs.index(row["Activity_ID"])])

# add semaphoreIN to activities and semaphores
for activity in activities:
    buf = activity.semaphoresIN
    activity.semaphoresIN = []
    while len(buf) > 0:
        semaphore_id = buf.pop(0)
        # if start equals '[' -> start of or)
        if semaphore_id[0] == '[':
            semaphore_id = semaphore_id[1:]
            
            if semaphore_id[-1] == ']':
                semaphore_id = semaphore_id[:-1]
                activity.semaphoresIN.append(semaphores[semaphore_IDs.index(semaphore_id)])
                semaphores[semaphore_IDs.index(semaphore_id)].activityIN = activity
                print(semaphore_id)
            else:
                semaphore_ids = []
                while semaphore_id[-1] != ']':
                    semaphore_ids.append(semaphore_id)
                    semaphore_id = buf.pop(0)
                semaphore_ids.append(semaphore_id[:-1])
                for semaphore_id in semaphore_ids:
                    activity.semaphoresIN.append(semaphores[semaphore_IDs.index(semaphore_id)])
                    semaphores[semaphore_IDs.index(semaphore_id)].activityIN = activity
                    # set the groupWith value of the semaphores and add every semaphore to the group except for the current one
                    for semaphore in semaphore_ids:
                        if not semaphore == semaphore_id:
                            semaphores[semaphore_IDs.index(semaphore_id)].groupWith.append(semaphores[semaphore_IDs.index(semaphore)])
                print(semaphore_ids)
        
        # if end equals ']' -> end of or)
        
        # else -> and
        else:
            activity.semaphoresIN.append(semaphores[semaphore_IDs.index(semaphore_id)])
            semaphores[semaphore_IDs.index(semaphore_id)].activityIN = activity
            print(semaphore_id)
            
import graphviz as gv
dot = gv.Digraph(comment='Flowchart')
dummyCounter = 0

# mmaybe pass ID of activity, semaphore that you want to color for the animation

ACTIVITY_RUNNING = 'green'
SEMAPHORE_ACTIVE = 'red'

def createRects(color, inverseColor):
    for activity in activities:
        color = color
        fontcolor = inverseColor
        if activity.currentValue > 0:
            color = ACTIVITY_RUNNING
            fontcolor = 'white'
        dot.node(str(activity.ID), shape='record', style='rounded,filled', label='{'+activity.parentTask.name+'|'+activity.name+'}', color = inverseColor, fontcolor=fontcolor, fillcolor=color)
        
def createMutexs(color, inverseColor):
    for mutex in mutexs:
        dot.node(str(mutex.ID), shape='polygon', sides='5', label=mutex.name)
        for activity in mutex.activities:
            dot.edge(str(mutex.ID), str(activity.ID), arrowhead='none', style='dashed', splines='polyline')
        
def createSemaphores(color, inverseColor):
    global dummyCounter
    
    # add color just like in createRects
    
    semaphores_buf = semaphores.copy()
    while len(semaphores_buf) != 0:
        
        semaphore = semaphores_buf.pop(0)
        
        color = SEMAPHORE_BLOCKING
        if semaphore.currentValue > 0:
            color = SEMAPHORE_ACTIVE
            
        # if semaphore has groupWith, connect those semaphores together. Those will then be erased from the list
        if len(semaphore.groupWith) != 0:
            
            # TODO: add initial Value to the semaphore
            
            # draw semaphores
            middleDummyName = "Dummy" + str(dummyCounter)
            dot.node(middleDummyName, shape='point', width="0.01", height="0.01", color=color)
            dummyCounter += 1
            
            # first the semaphore itself
            if semaphore.initialValue > 0:
                    # this is the edge with a point in the middle
                    dot.node("Dummy" + str(dummyCounter), shape='point', width="0.01", height="0.01", color=color)
                    dot.edge(str(semaphore.activityOUT.ID), "Dummy" + str(dummyCounter), arrowhead='none', splines='polyline', color=color)
                    dot.edge("Dummy" + str(dummyCounter), middleDummyName, label=semaphore.name, arrowhead='none', splines='polyline', color=color)
                    dummyCounter += 1
                    
                    # this is this init-thing (edge with point at the end)
                    dot.node("Dummy" + str(dummyCounter), shape='point', xlabel=(lambda valorem: '' if valorem == 1 else str(valorem))(semaphore.initialValue))
                    dot.edge("Dummy" + str(dummyCounter-1), "Dummy" + str(dummyCounter), arrowhead='none', splines='polyline')
                    dummyCounter += 1
            else:
                dot.edge(str(semaphore.activityOUT.ID), middleDummyName, label=semaphore.name, arrowhead='none', splines='polyline', color=color)
            
            # after the groupSemaphores
            colorOfLastSemaphore = color
            for groupSemaphore in semaphore.groupWith:
                if groupSemaphore.currentValue > 0:
                    color = SEMAPHORE_ACTIVE
                    colorOfLastSemaphore = SEMAPHORE_ACTIVE
                else:
                    color = SEMAPHORE_BLOCKING
                
                if groupSemaphore.initialValue > 0:
                    # this is the edge with a point in the middle
                    dot.node("Dummy" + str(dummyCounter), shape='point', width="0.01", height="0.01", color=color)
                    dot.edge(str(groupSemaphore.activityOUT.ID), "Dummy" + str(dummyCounter), arrowhead='none', splines='polyline', color=color)
                    dot.edge("Dummy" + str(dummyCounter), middleDummyName, label=groupSemaphore.name, arrowhead='none', splines='polyline', color=color)
                    dummyCounter += 1
                    
                    # this is this init-thing (edge with point at the end)
                    dot.node("Dummy" + str(dummyCounter), shape='point', xlabel=(lambda valorem: '' if valorem == 1 else str(valorem))(groupSemaphore.initialValue))
                    dot.edge("Dummy" + str(dummyCounter-1), "Dummy" + str(dummyCounter), arrowhead='none', splines='polyline')
                    dummyCounter += 1
                else:
                    dot.edge(str(groupSemaphore.activityOUT.ID), middleDummyName, label=groupSemaphore.name, arrowhead='none', splines='polyline', color=color)
                
                semaphores_buf.remove(groupSemaphore)
                
            dot.node(middleDummyName, shape='point', width="0.01", height="0.01", color=colorOfLastSemaphore)
            dot.edge(middleDummyName, str(semaphore.activityIN.ID), arrowhead='normal', splines='polyline', color=colorOfLastSemaphore)
            
        else:
            # if semaphore is leading to the same task where it came from, use another arrowhead
            if semaphore.activityIN.parentTask.ID == semaphore.activityOUT.parentTask.ID:
                # if semaphore has an initial value, connect an edge to the semaphore
                if semaphore.initialValue > 0:
                    dot.node("Dummy" + str(dummyCounter), shape='point', width="0.01", height="0.01", color=color)
                    dot.edge(str(semaphore.activityOUT.ID), "Dummy" + str(dummyCounter), arrowhead='none', splines='polyline', color=color)
                    dot.edge("Dummy" + str(dummyCounter), str(semaphore.activityIN.ID), label=semaphore.name, arrowhead='onormal', splines='polyline', color=color)
                    dummyCounter += 1
                    dot.node("Dummy" + str(dummyCounter), shape='point', xlabel=(lambda valorem: '' if valorem == 1 else str(valorem))(semaphore.initialValue))
                    dot.edge("Dummy" + str(dummyCounter-1), "Dummy" + str(dummyCounter), arrowhead='none', splines='polyline')
                    dummyCounter += 1
                else:
                    dot.edge(str(semaphore.activityOUT.ID), str(semaphore.activityIN.ID), label=semaphore.name, arrowhead='onormal', splines='polyline', color=color)
            else:
                if semaphore.initialValue > 0:
                    dot.node("Dummy" + str(dummyCounter), shape='point', width="0.01", height="0.01", color=color)
                    dot.edge(str(semaphore.activityOUT.ID), "Dummy" + str(dummyCounter), arrowhead='none', splines='polyline', color=color)
                    dot.edge("Dummy" + str(dummyCounter), str(semaphore.activityIN.ID), label=semaphore.name, splines='polyline', color=color)
                    dummyCounter += 1
                    dot.node("Dummy" + str(dummyCounter), shape='point', xlabel=(lambda valorem: '' if valorem == 1 else str(valorem))(semaphore.initialValue))
                    dot.edge("Dummy" + str(dummyCounter-1), "Dummy" + str(dummyCounter), arrowhead='none', splines='polyline')
                    dummyCounter += 1
                else:
                    dot.edge(str(semaphore.activityOUT.ID), str(semaphore.activityIN.ID), label=semaphore.name, splines='polyline', color=color)
            
historySemaphores = []

def getAllStartPoints():
    # get all activities that have no predecessor
    startPoints = []
    for semaphore in semaphores:
        if semaphore.initialValue > 0:
            startPoints.append(semaphore)
            
    return startPoints

def getActiveSemaphores():
    activeSemaphores = []
    for semaphore in semaphores:
        if semaphore.currentValue > 0:
            activeSemaphores.append(semaphore)
            
    return activeSemaphores

def getActiveActivities():
    activeActivities = []
    for activity in activities:
        if activity.currentValue > 0:
            activeActivities.append(activity)
            
    return activeActivities

def getNextFrame():
    # create a new image with highlighted nodes and edges (Tasks/Activities, Semaphores)
    flagFinished = False # if activity finished -> False
    startPoints = getAllStartPoints()
    
    # check if you can activate a new semaphore
    for startPoint in startPoints:
        if startPoint.currentValue <= 0:
            startPoint.currentValue = 1
            startPoint.initialValue -= 1
        # else: do nothing - wait for the semaphore to become free
        
    activeSemaphores = getActiveSemaphores()
        
    # check for the activities behind the active semaphores
    for activeSemaphore in activeSemaphores:
        # look, if the following activity is ready to run (all incoming semaphores == 1) or already running
        if activeSemaphore.activityIN.currentValue <= 0:
            # check if all incoming semaphores are active
            
            flag = True
            semaphoresIN = activeSemaphore.activityIN.semaphoresIN.copy()
            
            while flag and len(semaphoresIN) != 0:
                semaphoreIN = semaphoresIN.pop(0)
                # if there is a group, check if one of the semaphores is active
                if len(semaphoreIN.groupWith) != 0:
                    if semaphoreIN.currentValue <= 0:
                        for groupSemaphore in semaphoreIN.groupWith:
                            if groupSemaphore.currentValue > 0:
                                break
                            
                            if groupSemaphore == semaphoreIN.groupWith[-1]:
                                flag = False
                                break
                    if flag:
                        for groupSemaphore in semaphoreIN.groupWith:
                            semaphoresIN.remove(groupSemaphore)
                elif semaphoreIN.currentValue == 0:
                    flag = False
                    break
            
            if flag:
                activeSemaphore.activityIN.currentValue = 1
                
                
    activeActivities = getActiveActivities()
    
    for activeActivity in activeActivities:
        if activeActivity.currentDuration <= 0:
            # next steps: activate all semaphores going out of the activity
            semaphoresIN = activeActivity.semaphoresIN
            for semaphoreIN in semaphoresIN:
                semaphoreIN.currentValue = 0
            activeActivity.currentValue = 0
            semaphoresOUT = activeActivity.semaphoresOUT
            for semaphoreOUT in semaphoresOUT:
                semaphoreOUT.currentValue = 1
            flagFinished = True
            activeActivity.currentDuration = activeActivity.duration
        else:
            activeActivity.currentDuration -= 1
            
    if flagFinished:
        activeSemaphores = getActiveSemaphores()
        # check for the activities behind the active semaphores
        for activeSemaphore in activeSemaphores:
            # look, if the following activity is ready to run (all incoming semaphores == 1) or already running
            if activeSemaphore.activityIN.currentValue <= 0:
                # check if all incoming semaphores are active
                
                flag = True
                semaphoresIN = activeSemaphore.activityIN.semaphoresIN.copy()
                
                while flag and len(semaphoresIN) != 0:
                    semaphoreIN = semaphoresIN.pop(0)
                    # if there is a group, check if one of the semaphores is active
                    if len(semaphoreIN.groupWith) != 0:
                        if semaphoreIN.currentValue <= 0:
                            for groupSemaphore in semaphoreIN.groupWith:
                                if groupSemaphore.currentValue > 0:
                                    break
                                
                                if groupSemaphore == semaphoreIN.groupWith[-1]:
                                    flag = False
                                    break
                        if flag:
                            for groupSemaphore in semaphoreIN.groupWith:
                                semaphoresIN.remove(groupSemaphore)
                    elif semaphoreIN.currentValue == 0:
                        flag = False
                        break
                
                if flag:
                    activeSemaphore.activityIN.currentValue = 1
                    activeSemaphore.activityIN.currentDuration -= 1

def saveInitialValuesSemaphores():
    # save the initial values of all semaphores
    for semaphore in semaphores:
        if semaphore.initialValue > 0:
            historySemaphores.append([semaphore.ID, semaphore.initialValue])

def reinitializeInitialValuesSemaphores():
    # set all semaphores back to their initial values
    for valorem in historySemaphores:
        semaphores[semaphore_IDs.index(valorem[0])].initialValue = valorem[1]

# just a helper method
def createNextFrame(color='white', inverseColor='black'):
    getNextFrame()
    getCurrentImage(color, inverseColor)
    
def getCurrentImage(color='white', inverseColor='black'):
    global dot
    dot = gv.Digraph(comment='Flowchart')
    global dummyCounter
    dummyCounter = 0
    
    createRects(color, inverseColor)
    createMutexs(color, inverseColor)
    createSemaphores(color, inverseColor)
    dot.render('rectangle_arrow', view=True, format='svg')

['21']
['[12', '13]', '[14', '15]']
21
['12', '13']
['14', '15']


In [507]:
getCurrentImage()

In [513]:
createNextFrame()

In [388]:
activities
print("Hallo Welt")

Hallo Welt


In [13]:
import frontend.generator as gn

df = gn.openFromCSV(Path="dataTest.csv")

In [17]:
gn.getSingleImage('white', 'black', 1, True)

b'<?xml version="1.0" encoding="UTF-8" standalone="no"?>\r\n<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"\r\n "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">\r\n<!-- Generated by graphviz version 7.1.0 (20230121.1956)\r\n -->\r\n<!-- Pages: 1 -->\r\n<svg width="363pt" height="675pt"\r\n viewBox="0.00 0.00 363.04 675.32" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink">\r\n<g id="graph0" class="graph" transform="scale(1 1) rotate(0) translate(4 671.32)">\r\n<!-- Activitya1 -->\r\n<g id="node1" class="node">\r\n<title>Activitya1</title>\r\n<path fill="none" stroke="black" d="M83.5,-620.82C83.5,-620.82 126.5,-620.82 126.5,-620.82 132.5,-620.82 138.5,-626.82 138.5,-632.82 138.5,-632.82 138.5,-654.82 138.5,-654.82 138.5,-660.82 132.5,-666.82 126.5,-666.82 126.5,-666.82 83.5,-666.82 83.5,-666.82 77.5,-666.82 71.5,-660.82 71.5,-654.82 71.5,-654.82 71.5,-632.82 71.5,-632.82 71.5,-626.82 77.5,-620.82 83.5,-620.82"/>\r\n<text text-anchor="middle" x="105" y="-651

In [3]:
a = gn.getImages('white', 'black', 0, 0)

In [24]:
len(a)

1

In [3]:
gn.createGIF()

In [289]:
##### this is the first implementation of the input checker #####

from io import StringIO
import pandas as pd

class InputException(Exception):
    def __init__(self, msg):
        self.msg = msg

class InputChecker():
    columnTypes = {'Task_ID': 'str', 'Task_Name': 'str', 'Activity_ID': 'str', 'Activity_Name': 'str', 'Activity_Duration': 'int', 'Semaphore_ID': 'str|list(str)', 'Semaphore_Name': 'str|list(str)', 'Semaphore_Initial_Value': 'int|list(int)', 'Predecessor_Semaphore_ID': 'str|list(str)', 'Mutex_ID': 'str|list(str)', 'Mutex_Name': 'str|list(str)'}
    
    # this method checks if the input is valid with various different other methods
    def checkInput(self, input, toCheck=[]):
        df = None
        
        try:
            df = pd.read_csv(StringIO(input), sep=',')
            
            for func in toCheck:
                func(df)
                
        except InputException as ie:
            return False, ie.msg
        
        except Exception:
            return False, "It seems like the data provided is not in CSV format."
            
        return True, df
    
    def checkColumns(self, input):
        # check if the columns are correct

        input_columns = input.columns
        for column in self.columnTypes.keys():
            if column not in input_columns:
                raise InputException("The column '" + column + "' is missing.")
            else:
                input_columns = input_columns.drop(column)
        
        if not input_columns.empty:
            raise InputException("The column '" + input_columns[0] + "' is not needed.")
    
    def checkColumnTypes(self, input):
    
        for columnType in self.columnTypes:
            if self.columnTypes[columnType] == 'int':
                if not input[columnType].dtype == 'int64':
                    raise InputException("The column '" + columnType + "' is not of the correct type. It should be of type 'int'.")
            elif self.columnTypes[columnType] == 'int|list(int)':
                for row in input[columnType].str.split(';'):
                    for value in row:
                        try:
                            int(value)
                        except:
                            raise InputException("The column '" + columnType + "' is not of the correct type. It should be of type 'int' or 'list(int)'.")
    
    def checkEmptyCells(self, input):
        # check if there are empty cells
        for column in self.columnTypes.keys():
            if input[column].isnull().values.any():
                raise InputException("There are empty cells in the column '" + column + "'. Try using 'None' instead.")
    
    def checkSemaphores(self, input):
        predecessorSemaphores = self.__getAllPredecessorSemaphoreIDs(input)
        outgoingSemaphores = self.__getAllSemaphoreIDs(input)
        
        # check if the predecessor semaphores are in the list of semaphores
        # also have to consider OR-groups
        
        for semaphore in predecessorSemaphores:
            if semaphore not in outgoingSemaphores.values:
                raise InputException("The semaphore '" + semaphore + "' has no beginning.")
                    
        # check if the semaphores are in the list of predecessor semaphores
        
        for semaphore in outgoingSemaphores:
            if semaphore not in predecessorSemaphores.values:
                raise InputException("The semaphore '" + semaphore + "' has nowhere to go.")
        
    def checkForUniqueIDs(self, input):
        # check if the IDs are unique
        for column in ['Activity_ID', 'Semaphore_ID', 'Predecessor_Semaphore_ID']:
            if column == 'Predecessor_Semaphore_ID':
                predecessorSemaphores = self.__getAllPredecessorSemaphoreIDs(input)
                if len(predecessorSemaphores.unique()) != len(predecessorSemaphores):
                    raise InputException("Looks like some semaphores are used multiple times as predecessors. Check for column 'Predecessor_Semaphore_ID'.")
            elif column == 'Semaphore_ID':
                semaphores = self.__getAllSemaphoreIDs(input)
                if len(semaphores.unique()) != len(semaphores):
                    raise InputException("Looks like some semaphores are used multiple times as successors. Check for column 'Semaphore_ID'.")
            elif len(input[column].unique()) != len(input[column]):
                raise InputException("The IDs in the column '" + column + "' are not unique.")
            
    def test(self, input):
        return self.__getAllPredecessorSemaphoreIDs(input)
    
    def __getAllSemaphoreIDs(self, input):
        return input['Semaphore_ID'].str.split(';').apply(pd.Series).stack().reset_index(drop=True)
    
    def __getAllPredecessorSemaphoreIDs(self, input):
        return input['Predecessor_Semaphore_ID'].str.split(';').apply(pd.Series).stack().reset_index(drop=True).apply(lambda x: x[1:] if x[0] == '[' else (x[:-1] if x[-1] == ']' else x))

In [290]:
f = open("dataTest.csv", "r")
content = f.read()

df = pd.read_csv(StringIO(content), sep=',')

In [291]:
df

,Task_ID,Task_Name,Activity_ID,Activity_Name,Activity_Duration,Semaphore_ID,Semaphore_Name,Semaphore_Initial_Value,Predecessor_Semaphore_ID,Mutex_ID,Mutex_Name
0,1,Task1,1,Activity1,1,12;13;16,Semaphore12;Semaphore13;Semaphore16,0;0;0,51,None,None
1,2,Task2,2,Activity2,1,24,Semaphore24,0,12,234,Mutex234
2,3,Task3,3,Activity3,1,36,Semaphore36,0,13,234,Mutex234
3,4,Task4,4,Activity4,1,46,Semaphore46,0,24,234,Mutex234
4,5,Task5,5,Activity5a,1,55,Semaphore55,0,56;65,None,None
5,5,Task5,7,Activity5b,1,51;56,Semaphore51;Semaphore56,1;7,55,None,None
6,6,Task6,6,Activity6,1,65,Semaphore65,0,[36;46;16],None,None


In [292]:
ic = InputChecker()

ic.checkInput(content, [ic.checkColumns, ic.checkEmptyCells, ic.checkForUniqueIDs, ic.checkColumnTypes, ic.checkSemaphores])

(True,
    Task_ID Task_Name  Activity_ID Activity_Name  Activity_Duration  \
 0        1     Task1            1     Activity1                  1   
 1        2     Task2            2     Activity2                  1   
 2        3     Task3            3     Activity3                  1   
 3        4     Task4            4     Activity4                  1   
 4        5     Task5            5    Activity5a                  1   
 5        5     Task5            7    Activity5b                  1   
 6        6     Task6            6     Activity6                  1   
 
   Semaphore_ID                       Semaphore_Name Semaphore_Initial_Value  \
 0     12;13;16  Semaphore12;Semaphore13;Semaphore16                   0;0;0   
 1           24                          Semaphore24                       0   
 2           36                          Semaphore36                       0   
 3           46                          Semaphore46                       0   
 4           55        